# Data splits

This notebook focuses on building the data splits to train the model, namely: train, validation and test.

*Constraint: every splits must contain unique subjects, namely different MR session for a subject x, needs to be contained in only ONE split*.

In [38]:
import pandas as pd
import os
import sys

# Append the path to access custom modules
if '../' not in sys.path:
    sys.path.append('../')

import src.utils.data_splitter as ds
import src.utils.dataset_helper as db
import importlib

In [39]:
importlib.reload(ds);
importlib.reload(db);

In [40]:
df = pd.read_csv('../data/csv/final_dataset.csv')
df = df.drop(columns='Unnamed: 0')

In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2379 entries, 0 to 2378
Data columns (total 11 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Subject                       2379 non-null   object 
 1   MR_session                    2379 non-null   object 
 2   TOTAL_HIPPOCAMPUS_VOLUME      2379 non-null   float64
 3   Left-Hippocampus_volume       2379 non-null   float64
 4   lh_parahippocampal_thickness  2379 non-null   float64
 5   lh_parahippocampal_volume     2379 non-null   int64  
 6   rh_parahippocampal_volume     2379 non-null   int64  
 7   rh_parahippocampal_thickness  2379 non-null   float64
 8   Right-Hippocampus_volume      2379 non-null   float64
 9   label                         2379 non-null   object 
 10  CDRTOT                        2379 non-null   float64
dtypes: float64(6), int64(2), object(3)
memory usage: 204.6+ KB


In [42]:
helper = db.DatasetHelper

## Undersample majority class

- Class distribution before resample

In [43]:
df['label'].value_counts()

label
Cognitevely-normal    1892
Early-stage            368
Demented               119
Name: count, dtype: int64

In [44]:
df = df.drop(index=df[df['Subject'] == 'OAS31168'].index)
df = df.drop(index=df[df['Subject'] == 'OAS30530'].index)

- Get the undersampled dataset that will be the basis dataset from now on

In [45]:
df = df.loc[helper.undersample_majority_class(df['label'], 550)]

- Class distribution after resample

In [46]:
df['label'].value_counts()

label
Cognitevely-normal    550
Early-stage           367
Demented              119
Name: count, dtype: int64

- Create the csv file

In [47]:
df.to_csv('../data/csv/final_dataset_undersampled.csv', index=False)

- At this point we can create the new folder with the freesurfers whose MR label is in the resampled df

In [48]:
helper.move_folders(
    subject_experiments=df['MR_session'].to_list(), 
    base_dir=os.path.join('..', 'data', 'freesurfers'),
    dst_dir=os.path.join('..', 'data', 'freesurfers_undersampled')
)

## Define the test set

In [49]:
df = pd.read_csv('../data/csv/final_dataset_undersampled.csv')

In [50]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1036 entries, 0 to 1035
Data columns (total 11 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Subject                       1036 non-null   object 
 1   MR_session                    1036 non-null   object 
 2   TOTAL_HIPPOCAMPUS_VOLUME      1036 non-null   float64
 3   Left-Hippocampus_volume       1036 non-null   float64
 4   lh_parahippocampal_thickness  1036 non-null   float64
 5   lh_parahippocampal_volume     1036 non-null   int64  
 6   rh_parahippocampal_volume     1036 non-null   int64  
 7   rh_parahippocampal_thickness  1036 non-null   float64
 8   Right-Hippocampus_volume      1036 non-null   float64
 9   label                         1036 non-null   object 
 10  CDRTOT                        1036 non-null   float64
dtypes: float64(6), int64(2), object(3)
memory usage: 89.2+ KB


- Instantiate the data splitter

In [51]:
splitter = ds.DataSplitterTrainTest(df['MR_session'], df['label'])

- Retrieve the the train and test instances

In [52]:
train_idx, test_idx = splitter.get_train_test_set_idx()

- Check train label distribution

In [53]:
df.loc[train_idx]['label'].value_counts()

label
Cognitevely-normal    479
Early-stage           326
Demented              102
Name: count, dtype: int64

- Check test label distribution

In [54]:
df.loc[test_idx]['label'].value_counts()

label
Cognitevely-normal    71
Early-stage           41
Demented              17
Name: count, dtype: int64

- Create a csv file related to test instances

In [55]:
df.loc[test_idx].to_csv('../data/csv/test.csv')

- Test volumetric images can now be moved in the dedicated folder

In [56]:
helper.move_folders(
    subject_experiments=df.loc[test_idx, 'MR_session'],
    base_dir=os.path.join('..', 'data', 'freesurfers_undersampled'),
    dst_dir=os.path.join('..', 'data', 'test_freesurfers')
)

## Define validation set

We can procede like before to define the validation set, to evaluate the model during training

In [57]:
# Work on a copy of the training set only
X_train = df.loc[train_idx].copy(deep=True) 

- Split the data

In [58]:
splitter = ds.DataSplitterTrainTest(X_train['MR_session'], X_train['label'])
train_idx, val_idx = splitter.get_train_test_set_idx()

In [59]:
X_train.loc[train_idx, 'label'].value_counts()

label
Cognitevely-normal    423
Early-stage           293
Demented               87
Name: count, dtype: int64

In [60]:
X_train.loc[val_idx, 'label'].value_counts()

label
Cognitevely-normal    56
Early-stage           33
Demented              15
Name: count, dtype: int64

- Move train set images to a separate folder

In [61]:
helper.move_folders(
    subject_experiments=X_train.loc[train_idx, 'MR_session'],
    base_dir=os.path.join('..', 'data', 'freesurfers_undersampled'),
    dst_dir=os.path.join('..', 'data', 'train_freesurfers')
)

- Move the validation set images to a separate folder

In [62]:
helper.move_folders(
    subject_experiments=X_train.loc[val_idx, 'MR_session'],
    base_dir=os.path.join('..', 'data', 'freesurfers_undersampled'),
    dst_dir=os.path.join('..', 'data', 'validation_freesurfers')
)

- Create clean (with no augmentated instances) csv file related to train and validation set

In [63]:
X_train.loc[train_idx].to_csv('../data/csv/train_clean.csv')
X_train.loc[val_idx].to_csv('../data/csv/validation_clean.csv')

## Add augmentation labels to train set

- This section will add to the train dataset (train + validation) the augmented instances to improve class balance
- Let's start by augmenting MCIs

In [64]:
mci_aug_train = helper.augment_dataframe(
    df=X_train.loc[train_idx][X_train.loc[train_idx, 'label'] == 'Early-stage'].sample(n=30, random_state=42),
    id_col_name='MR_session',
    suffix_list=['flip']
) + helper.augment_dataframe(
    df=X_train.loc[train_idx][X_train.loc[train_idx, 'label'] == 'Early-stage'].sample(n=30, random_state=43),
    id_col_name='MR_session',
    suffix_list=['rot30']
) + helper.augment_dataframe(
    df=X_train.loc[train_idx][X_train.loc[train_idx, 'label'] == 'Early-stage'].sample(n=35, random_state=44),
    id_col_name='MR_session',
    suffix_list=['rot60']
) + helper.augment_dataframe(
    df=X_train.loc[train_idx][X_train.loc[train_idx, 'label'] == 'Early-stage'].sample(n=35, random_state=45),
    id_col_name='MR_session',
    suffix_list=['rot90']
)

- Augment demented label

In [65]:
dem_aug_train = helper.augment_dataframe(
    df=X_train.loc[train_idx][X_train.loc[train_idx, 'label'] == 'Demented'].sample(n=81, random_state=42),
    id_col_name='MR_session',
    suffix_list=['flip']
) + helper.augment_dataframe(
    df=X_train.loc[train_idx][X_train.loc[train_idx, 'label'] == 'Demented'].sample(n=87, random_state=43),
    id_col_name='MR_session',
    suffix_list=['rot30']
) + helper.augment_dataframe(
    df=X_train.loc[train_idx][X_train.loc[train_idx, 'label'] == 'Demented'].sample(n=87, random_state=44),
    id_col_name='MR_session',
    suffix_list=['rot60']
) + helper.augment_dataframe(
    df=X_train.loc[train_idx][X_train.loc[train_idx, 'label'] == 'Demented'].sample(n=81, random_state=45),
    id_col_name='MR_session',
    suffix_list=['rot90']
)

- Get the final training dataframe

In [66]:
X_train_final = pd.concat(objs=[X_train.loc[train_idx]] + mci_aug_train + dem_aug_train, ignore_index=True)

- This is the final distribution of the labels on the training set

In [67]:
X_train_final['label'].value_counts()

label
Cognitevely-normal    423
Early-stage           423
Demented              423
Name: count, dtype: int64

- Augment the validation set for the early-stage instances

In [68]:
mci_aug_val = helper.augment_dataframe(
    df=X_train.loc[val_idx][X_train.loc[val_idx, 'label'] == 'Early-stage'].sample(n=23, random_state=42),
    id_col_name='MR_session',
    suffix_list=['rot30']
)

- Augment the validation set for the demented instances

In [69]:
dem_aug_val = helper.augment_dataframe(
    df=X_train.loc[val_idx][X_train.loc[val_idx, 'label'] == 'Demented'].sample(n=11, random_state=42),
    id_col_name='MR_session',
    suffix_list=['flip']
) + helper.augment_dataframe(
    df=X_train.loc[val_idx][X_train.loc[val_idx, 'label'] == 'Demented'].sample(n=10, random_state=43),
    id_col_name='MR_session',
    suffix_list=['rot30']
) + helper.augment_dataframe(
    df=X_train.loc[val_idx][X_train.loc[val_idx, 'label'] == 'Demented'].sample(n=10, random_state=44),
    id_col_name='MR_session',
    suffix_list=['rot60']
) + helper.augment_dataframe(
    df=X_train.loc[val_idx][X_train.loc[val_idx, 'label'] == 'Demented'].sample(n=10, random_state=45),
    id_col_name='MR_session',
    suffix_list=['rot90']
)

- Get the final validation dataset

In [70]:
X_val_final = pd.concat(objs=[X_train.loc[val_idx]] + mci_aug_val + dem_aug_val, ignore_index=True)

- This is the final distribution of labels on the validation set

In [71]:
X_val_final['label'].value_counts()

label
Early-stage           56
Cognitevely-normal    56
Demented              56
Name: count, dtype: int64

- Build a new column to indicate the augmentation operation to perform for a specific instance

In [72]:
X_train_final['augmentation'] = helper.extract_augmentation_column(X_train_final['MR_session'])
X_val_final['augmentation'] = helper.extract_augmentation_column(X_val_final['MR_session'])

In [73]:
X_train_final['MR_session_original'] = X_train_final['MR_session_original'].fillna(value='normal')
X_val_final['MR_session_original'] = X_val_final['MR_session_original'].fillna(value='normal')

- Save the augmented dataset to csv

In [74]:
X_train_final.to_csv('../data/csv/train.csv')
X_val_final.to_csv('../data/csv/validation.csv')